## Battle of Neighborhood - The Preparation

### Step 1: Import important Libraries that will be required entire Project
#### Here, we are considering to continue the complete project in same Noteb

In [1]:
# Import all required libraries
import numpy as np # To handle data in a vectorize manner

import pandas as pd # To get the data in DataFrame

import json # To get and read the JSON file

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # To convert an address into latitude and longitude values

import requests # To handle JSON requests

from pandas.io.json import json_normalize # To conver json file data into DataFrame

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # To create cluster by K-Means 

!conda install -c conda-forge folium=0.5.0 --yes
import folium # Map rendering library

from bs4 import BeautifulSoup # Install and import BeautifulSoup4 for scraping the Wikipedia page and get the table in csv format

import lxml.html as lh
import re

print('Libraries imported')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

### Step 2: Read the table from Wikipedia page and get the table data into pandas DataFrame

In [2]:
# Read URL
input_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Fetch the content from the url, using the requests library
wiki_data = requests.get(input_url)

# Used 'lxml' parser to parse the url content and store it in a variable.
wiki_content = BeautifulSoup(wiki_data.content, "lxml")

table_data = wiki_content.find("table", class_="wikitable sortable")

# Create DataFrame
df_columns = []
df_rows = []

for i in table_data.find_all('th'):
    df_columns.append(re.split("\n", i.text)[0])

table_rows = table_data.find_all('tr')

for tr in table_rows:
    # Fetch Rows
    td = tr.find_all('td')
    row = [re.split("\n", tr.text)[0] for tr in td]
    df_rows.append(row)

PostalCode_df = pd.DataFrame(df_rows[1:],columns=df_columns)
PostalCode_df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### Step 3: Drop the rows with value as 'Not assigned' in Borough

In [8]:
# Drop rows with "Not assigned"
T_df = PostalCode_df[~PostalCode_df['Borough'].isin(['Not assigned'])]
Toronto_df = pd.DataFrame(T_df.groupby(['Postcode', 'Borough'],sort=False)['Neighbourhood'].apply(','.join)).reset_index()
Toronto_df.rename(columns={'Postcode':'PostalCode','Neighbourhood':'Neighborhood'}, inplace=True)
Toronto_df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [4]:
# Display shape of the DataFrame
Toronto_df.shape

(103, 3)